In [80]:
import pandas as pd
from zipline.pipeline import Pipeline
from zipline.pipeline.data import USEquityPricing
from sharadar.pipeline.factors import (
    MarketCap,
    EV,
    Fundamentals
)
from sharadar.pipeline.engine import load_sharadar_bundle, symbol, symbols, make_pipeline_engine
from zipline.pipeline.filters import StaticAssets
import time
import datetime
import math
from zipline.utils.calendars import get_calendar
from pandas.tseries.offsets import DateOffset
import numpy as np
from zipline.pipeline.factors import CustomFactor, Returns
from sharadar.pipeline.engine import WithAssetFinder

In [23]:
np.set_printoptions(suppress=True)

In [43]:
self = load_sharadar_bundle()

In [4]:
symbols(['IBM', 'F', 'AAPL'])

[Equity(199623 [IBM]), Equity(199713 [F]), Equity(199059 [AAPL])]

In [81]:
assets = [199059, 199623, 199713]
today = pd.to_datetime('2020-06-01', utc=True)
l = 1

In [82]:
assets_t = bundle.asset_finder.get_fundamentals(assets, 'assets_art', today, n=l)
assets_t[0][0] 
#2020-03-31 320,400.00

320400000000.0

In [83]:
assets_t = bundle.asset_finder.get_fundamentals(assets, 'assets_arq', today, n=(l+4))
assets_t[0][0] 
#2019-03-31 341,998.00

341998000000.0

In [84]:
#5 years
l = 20
ta = self.asset_finder.get_fundamentals_df_window_length(assets, 'assets_art', today, l + 4)
ta

array([[3.20400e+11, 1.53403e+11, 2.64150e+11],
       [3.40618e+11, 1.52186e+11, 2.58537e+11],
       [3.38516e+11, 1.49620e+11, 2.58157e+11],
       [3.22239e+11, 1.54652e+11, 2.62184e+11],
       [3.41998e+11, 1.30926e+11, 2.63281e+11],
       [3.73719e+11, 1.23382e+11, 2.56540e+11],
       [3.65725e+11, 1.21990e+11, 2.58966e+11],
       [3.49197e+11, 1.21622e+11, 2.58079e+11],
       [3.67502e+11, 1.25285e+11, 2.67230e+11],
       [4.06794e+11, 1.25356e+11, 2.57808e+11],
       [3.75319e+11, 1.21636e+11, 2.51273e+11],
       [3.45173e+11, 1.20495e+11, 2.47469e+11],
       [3.34532e+11, 1.17495e+11, 2.44094e+11],
       [3.31141e+11, 1.17470e+11, 2.37951e+11],
       [3.21686e+11, 1.15606e+11, 2.34963e+11],
       [3.05602e+11, 1.18056e+11, 2.39678e+11],
       [3.05277e+11, 1.18856e+11, 2.37288e+11],
       [2.93284e+11, 1.10495e+11, 2.24925e+11],
       [2.90479e+11, 1.08649e+11, 2.19431e+11],
       [2.73151e+11, 1.12729e+11, 2.16045e+11],
       [2.61194e+11, 1.12037e+11, 2.1237

In [89]:
ta_log = np.log(ta)
ta_log

array([[26.49283605, 25.75633428, 26.29978296],
       [26.55402745, 25.74836929, 26.27830465],
       [26.5478372 , 25.73136458, 26.27683376],
       [26.49855934, 25.76444327, 26.29231238],
       [26.55807073, 25.59789812, 26.29648774],
       [26.64677002, 25.53855107, 26.27055043],
       [26.62514752, 25.52720491, 26.27996262],
       [26.57890207, 25.52418371, 26.27653158],
       [26.6299946 , 25.55385698, 26.31137555],
       [26.73157275, 25.55442353, 26.27548096],
       [26.65104217, 25.52429882, 26.24980583],
       [26.56731158, 25.5148741 , 26.23455116],
       [26.53599838, 25.48966162, 26.22081923],
       [26.5258101 , 25.48944882, 26.19533061],
       [26.49684175, 25.47345369, 26.18269389],
       [26.44554944, 25.49442493, 26.20256219],
       [26.4444854 , 25.50117851, 26.19254043],
       [26.40440726, 25.42823611, 26.13903285],
       [26.39479712, 25.41138834, 26.11430367],
       [26.33329059, 25.44825255, 26.09875256],
       [26.28852926, 25.44209501, 26.081

In [91]:
def shift(arr, num, fill_value=np.nan):
    result = np.empty_like(arr)
    if num > 0:
        result[:num] = fill_value
        result[num:] = arr[:-num]
    elif num < 0:
        result[num:] = fill_value
        result[:num] = arr[-num:]
    else:
        result[:] = arr
    return result

ta_log_4 = shift(ta_log, -4)
ta_log_4

array([[26.55807073, 25.59789812, 26.29648774],
       [26.64677002, 25.53855107, 26.27055043],
       [26.62514752, 25.52720491, 26.27996262],
       [26.57890207, 25.52418371, 26.27653158],
       [26.6299946 , 25.55385698, 26.31137555],
       [26.73157275, 25.55442353, 26.27548096],
       [26.65104217, 25.52429882, 26.24980583],
       [26.56731158, 25.5148741 , 26.23455116],
       [26.53599838, 25.48966162, 26.22081923],
       [26.5258101 , 25.48944882, 26.19533061],
       [26.49684175, 25.47345369, 26.18269389],
       [26.44554944, 25.49442493, 26.20256219],
       [26.4444854 , 25.50117851, 26.19254043],
       [26.40440726, 25.42823611, 26.13903285],
       [26.39479712, 25.41138834, 26.11430367],
       [26.33329059, 25.44825255, 26.09875256],
       [26.28852926, 25.44209501, 26.08161471],
       [26.29120568, 25.48997647, 26.06333437],
       [26.169309  , 25.50164115, 26.0648103 ],
       [26.12828282, 25.54607646, 26.07486327],
       [        nan,         nan,       

In [92]:
ta_log - ta_log_4

array([[-0.06523467,  0.15843617,  0.00329522],
       [-0.09274256,  0.20981822,  0.00775422],
       [-0.07731033,  0.20415967, -0.00312885],
       [-0.08034273,  0.24025956,  0.01578081],
       [-0.07192387,  0.04404114, -0.01488781],
       [-0.08480274, -0.01587246, -0.00493052],
       [-0.02589465,  0.0029061 ,  0.03015678],
       [ 0.01159049,  0.00930962,  0.04198042],
       [ 0.09399622,  0.06419536,  0.09055631],
       [ 0.20576265,  0.06497471,  0.08015035],
       [ 0.15420042,  0.05084512,  0.06711194],
       [ 0.12176214,  0.02044917,  0.03198897],
       [ 0.09151298, -0.0115169 ,  0.0282788 ],
       [ 0.12140284,  0.06121271,  0.05629776],
       [ 0.10204463,  0.06206536,  0.06839022],
       [ 0.11225885,  0.04617238,  0.10380964],
       [ 0.15595613,  0.0590835 ,  0.11092572],
       [ 0.11320158, -0.06174037,  0.07569848],
       [ 0.22548812, -0.09025281,  0.04949337],
       [ 0.20500777, -0.09782391,  0.02388928],
       [        nan,         nan,       

In [94]:
x = np.flip(ta_log - ta_log_4, axis=0)
x

array([[        nan,         nan,         nan],
       [        nan,         nan,         nan],
       [        nan,         nan,         nan],
       [        nan,         nan,         nan],
       [ 0.20500777, -0.09782391,  0.02388928],
       [ 0.22548812, -0.09025281,  0.04949337],
       [ 0.11320158, -0.06174037,  0.07569848],
       [ 0.15595613,  0.0590835 ,  0.11092572],
       [ 0.11225885,  0.04617238,  0.10380964],
       [ 0.10204463,  0.06206536,  0.06839022],
       [ 0.12140284,  0.06121271,  0.05629776],
       [ 0.09151298, -0.0115169 ,  0.0282788 ],
       [ 0.12176214,  0.02044917,  0.03198897],
       [ 0.15420042,  0.05084512,  0.06711194],
       [ 0.20576265,  0.06497471,  0.08015035],
       [ 0.09399622,  0.06419536,  0.09055631],
       [ 0.01159049,  0.00930962,  0.04198042],
       [-0.02589465,  0.0029061 ,  0.03015678],
       [-0.08480274, -0.01587246, -0.00493052],
       [-0.07192387,  0.04404114, -0.01488781],
       [-0.08034273,  0.24025956,  0.015

In [98]:
x[4:]

array([[ 0.20500777, -0.09782391,  0.02388928],
       [ 0.22548812, -0.09025281,  0.04949337],
       [ 0.11320158, -0.06174037,  0.07569848],
       [ 0.15595613,  0.0590835 ,  0.11092572],
       [ 0.11225885,  0.04617238,  0.10380964],
       [ 0.10204463,  0.06206536,  0.06839022],
       [ 0.12140284,  0.06121271,  0.05629776],
       [ 0.09151298, -0.0115169 ,  0.0282788 ],
       [ 0.12176214,  0.02044917,  0.03198897],
       [ 0.15420042,  0.05084512,  0.06711194],
       [ 0.20576265,  0.06497471,  0.08015035],
       [ 0.09399622,  0.06419536,  0.09055631],
       [ 0.01159049,  0.00930962,  0.04198042],
       [-0.02589465,  0.0029061 ,  0.03015678],
       [-0.08480274, -0.01587246, -0.00493052],
       [-0.07192387,  0.04404114, -0.01488781],
       [-0.08034273,  0.24025956,  0.01578081],
       [-0.07731033,  0.20415967, -0.00312885],
       [-0.09274256,  0.20981822,  0.00775422],
       [-0.06523467,  0.15843617,  0.00329522]])